In [1]:
# import pkgutil
# if not pkgutil.find_loader("dreal"):
#   !curl https://raw.githubusercontent.com/dreal/dreal4/master/setup/ubuntu/22.04/install.sh | bash
#   !pip install dreal --upgrade
# if not pkgutil.find_loader("dreal"):
#   !pip install control

### Utility functions for pytorch and dreal

In [2]:
import torch
import numpy as np
import dreal as d


device = 'cuda' if torch.cuda.is_available() else 'cpu'

# pytorch utils
def torch_to_np(x):
    return x.cpu().detach().numpy()

def np_to_torch(x):
	return torch.from_numpy(x).float().to(device)

def pipe(x, *funcs):
	for f in funcs:
		x = f(x)
	return x

# dreal utils


def dreal_var(n, name='x'):
    return np.array([d.Variable("%s%d" % (name, i)) for i in range(n)])

def dreal_elementwise(x, func):
    """
    :param x: np.array[dreal.Variable]
    :param func: dreal function, for example, dreal.tanh
    """
    return np.array([
        func(x[i]) for i in range(len(x))
    ])

def dreal_sigmoid(x):
    return 1 / (1 + d.exp(-x))

### Dynamical System Benchmarks

In [3]:
import numpy as np
import torch
import matplotlib.pyplot as plt


def rk4_step(f, x, u, dt):
	""" RK4 simulation for continuous-time system """
	f1 = f(x, u)
	f2 = f(x + 0.5 * dt * f1, u)
	f3 = f(x + 0.5 * dt * f2, u)
	f4 = f(x + dt * f3, u)
	return x + (dt / 6.0) * (f1 + 2 * f2 + 2 * f3 + f4)
    
class Benchmark():
    def __init__(self) -> None:
        super().__init__()
        self.name = None
        self.nx = None
        self.nu = None
        self.lb: np.array = None
        self.ub: np.array = None
        self.init_control: np.array = None  # [nu, nx]
    
    def f_np(self, x, u): 
        """"
        :param x: np.array, [batch, nx]
        :param u: np.array [batch, nu]
        """
        pass
    
    def f_torch(self, x, u): 
        """"
        :param x: torch.tensor, [batch, nx]
        :param u: torch.tensor [batch, nu]
        """
        pass
    
    def f_dreal(self, x, u): 
        """
        :param x: np.array[dreal.Variable], [2,]
        :param u: np.array[dreal.Variable], [1,]
        """
        pass
    
    def in_domain_dreal(self, x, scale=1.):
        """
        :param x: np.array[dreal.Variable], [2,]
        """
        return d.And(
            x[0] >= self.lb[0] * scale,
            x[0] <= self.ub[0] * scale,
            x[1] >= self.lb[1] * scale,
            x[1] <= self.ub[1] * scale
        )
    
    def on_boundry_dreal(self, x, scale=2.):
        condition1 = d.And(
            x[0] >= self.lb[0] * scale * 0.99,
            x[0] <= self.ub[0] * scale * 0.99,
            x[1] >= self.lb[1] * scale * 0.99,
            x[1] <= self.ub[1] * scale * 0.99
        )
        condition2 = d.Not(
            d.And(
                x[0] >= self.lb[0] * scale * 0.97,
                x[0] <= self.ub[0] * scale * 0.97,
                x[1] >= self.lb[1] * scale * 0.97,
                x[1] <= self.ub[1] * scale * 0.97
            )
        )
        return d.And( condition1, condition2 )
    
    def sample_in_domain(self, n, scale=1.): pass
    
    def sample_out_of_domain(self, n, scale=1.): pass
    
    
class DoubleIntegrator(Benchmark):
    def __init__(self):
        super().__init__()
        self.name='dint'
        self.nx = 2
        self.nu = 1
        self.lb = np.array([-4., -4.])
        self.ub = np.array([4., 4.])
        self.init_control = np.zeros([self.nu, self.nx])
    
    def f_np(self, x, u):
        """
        :param x: [batch, 2], np.array
            x[:,0] = \theta
            x[:,1] = \dot\theta
        :param u: 
        :return: 
        """
        return np.stack([x[:, 1], u[:,0]], axis=1)
    
    def f_torch(self, x, u):
        """
        :param x: [batch, 2], torch.tensor
          x[:,0] = \theta
          x[:,1] = \dot\theta
        :param u: [batch, 1], torch.tensor
        """
        return torch.stack([x[:, 1], u[:,0]], dim=1)
    
    def f_dreal(self, x, u):
        """
        :param x: np.array[dreal.Variable], [2,]
        :param u: np.array[dreal.Variable], [1,]
        """
        return np.stack([x[1], u[0]], axis=0)
    
    def sample_in_domain(self, n, scale=1.):
        return np.random.uniform(self.lb * scale, self.ub * scale, (n, self.nx))
    
    def sample_out_of_domain(self, n, scale=1.):
        x = np.random.uniform(-1, 1, (n, self.nx))
        xnorm = np.maximum( np.abs(x[:, 0]) / (self.ub[0] * scale),  np.abs(x[:, 1]) / (self.ub[1] * scale) )
        x = x / xnorm[:, None]
        noise = np.random.uniform(0, 0.5, (n, self.nx))
        x = x + np.sign(x) * noise
        return x
    
class VanderPol(Benchmark):
    def __init__(self, mu = 1.):
        super().__init__()
        self.name = 'vanderpol'
        self.nx = 2
        self.nu = 1
        self.lb = np.array([-2., -2.])
        self.ub = np.array([2., 2.])
        self.init_control = np.array([[0, 1.]])
        self.mu = mu
        
    def f_np(self, x, u):
        """
        :param x: [batch, 2], np.array
        :param u: 
        :return: 
        """
        x1 = x[:,0]
        x2 = x[:, 1]

        x1d = x2
        x2d =  - x1 + self.mu * (1 - x1 ** 2) * x2 + u[:,0]

        return np.stack([x1d, x2d], axis=1)
    
    def f_torch(self, x, u):
        """
        :param x: [batch, 2], torch.tensor
          x[:,0] = \theta
          x[:,1] = \dot\theta
        :param u: [batch, 1], torch.tensor
        """
        x1 = x[:,0]
        x2 = x[:, 1]

        x1d = x2
        x2d =  - x1 + self.mu * (1 - x1 ** 2) * x2 + u[:,0]

        return torch.stack([x1d, x2d], dim=1)
    
    def f_dreal(self, x, u):
        """
        :param x: np.array[dreal.Variable], [2,]
        :param u: np.array[dreal.Variable], [1,]
        """
        x1 = x[0]
        x2 = x[1]

        x1d = x2
        x2d =  - x1 + self.mu * (1 - x1 ** 2) * x2 + u[0]

        return np.array([x1d, x2d])
    
    def sample_in_domain(self, n, scale=1.):
        return np.random.uniform(self.lb * scale, self.ub * scale, (n, self.nx))
    
    def sample_out_of_domain(self, n, scale=1.):
        x = np.random.uniform(-1, 1, (n, self.nx))
        xnorm = np.maximum( np.abs(x[:, 0]) / (self.ub[0] * scale),  np.abs(x[:, 1]) / (self.ub[1] * scale) )
        x = x / xnorm[:, None]
        noise = np.random.uniform(0, 0.5, (n, self.nx))
        x = x + np.sign(x) * noise
        return x     
    
    

class InvertedPendulum(Benchmark):
    def __init__(self):
        super().__init__()
        self.name = 'pendulum'
        self.nx = 2
        self.nu = 1
        self.lb = np.array([-2., -4.])
        self.ub = np.array([2., 4.])
        self.init_control = np.array([[-23.58639732,  -5.31421063]])
        
        self.G = 9.81  # gravity
        self.L = 0.5  # length of the pole
        self.m = 0.15  # ball mass
        self.b = 0.1  # friction
        
    def f_np(self, x, u):
        """
        :param x: [batch, 2], np.array
            x[:,0] = \theta
            x[:,1] = \dot\theta
        :param u: [batch,], np.array
        """
        theta = x[:,0]
        thetad = x[:,1]
        
        thetadd = self.G / self.L * np.sin(theta) - self.b / (self.m * self.L**2) * thetad + u[:,0] / (self.m * self.L**2)
        return np.stack([thetad, thetadd], axis=1)
    
    def f_torch(self, x, u):
        """
        :param x: [batch, 2], torch.tensor
            x[:,0] = \theta
            x[:,1] = \dot\theta
        :param u: [batch,], torch.tensor
        """
        theta = x[:,0]
        thetad = x[:,1]
        
        thetadd = self.G / self.L * torch.sin(theta) - self.b / (self.m * self.L**2) * thetad + u[:,0] / (self.m * self.L**2)
        return torch.stack([thetad, thetadd], dim=1)
    
    def f_dreal(self, x, u):
        """
        :param x: np.array[dreal.Variable], [2,]
        :param u: np.array[dreal.Variable], [1,] 
        """
        theta = x[0]
        thetad = x[1]
        thetadd = self.G / self.L * d.sin(theta) - self.b / (self.m * self.L**2) * thetad + u[0] / (self.m * self.L**2)
        return np.array([thetad, thetadd])
        
    def sample_in_domain(self, n, scale=1.):
        return np.random.uniform(self.lb * scale, self.ub * scale, (n, self.nx))
    
    def sample_out_of_domain(self, n, scale=1.):
        x = np.random.uniform(-1, 1, (n, self.nx))
        xnorm = np.maximum( np.abs(x[:, 0]) / (self.ub[0] * scale),  np.abs(x[:, 1]) / (self.ub[1] * scale) )
        x = x / xnorm[:, None]
        noise = np.random.uniform(0, 0.5, (n, self.nx))
        x = x + np.sign(x) * noise
        return x
    
class BicycleTracking(Benchmark):
    def __init__(self):
        super().__init__()
        self.name = 'tracking'
        self.v = 6.
        self.l = 1.
        self.nx = 2
        self.nu = 1
        
        self.lb = np.array([-0.7, -0.7])
        self.ub = np.array([0.7, 0.7])
        self.init_control = np.array([[-0.8471 , -1.6414]])
        
    def f_np(self, x, u):
        y1 = self.v * np.sin(x[:,1])
        y2 = self.v * np.tan(u[:,0]) / self.l - np.cos(x[:,1]) / (1 - x[:,0])
        return np.stack([y1, y2], axis=-1)
    
    def f_torch(self, x, u):
        y1 = self.v * torch.sin(x[:,1])
        y2 = self.v * torch.tan(u[:,0]) / self.l - torch.cos(x[:,1]) / (1 - x[:,0])
        return torch.stack([y1, y2], dim=-1)
    
    def f_dreal(self, x, u):
        y1 = self.v * d.sin(x[1])
        y2 = self.v * d.tan(u[0]) / self.l - d.cos(x[1]) / (1 - x[0])
        return np.array([y1, y2])
    
    def sample_in_domain(self, n, scale=1.):
        return np.random.uniform(self.lb * scale, self.ub * scale, (n, self.nx))
    
    def sample_out_of_domain(self, n, scale=1.):
        x = np.random.uniform(-1, 1, (n, self.nx))
        xnorm = np.maximum( np.abs(x[:, 0]) / (self.ub[0] * scale),  np.abs(x[:, 1]) / (self.ub[1] * scale) )
        x = x / xnorm[:, None]
        noise = np.random.uniform(0, 0.1, (n, self.nx))
        x = x + np.sign(x) * noise
        return x
        
        

In [ ]:
def test_dreal_for_benchmark():
    sys = DoubleIntegrator()
    x_dreal = dreal_var(sys.nx, name='x')
    u_dreal = dreal_var(sys.nu, name='u')
    xd_dreal = sys.f_dreal(x_dreal, u_dreal)

    x = np.random.randn(1, sys.nx)
    u = np.random.randn(1, sys.nu)
    xd = sys.f_np(x, u)[0]
    print('x:', x)
    print('u:', u)
    print('xd:', xd)

    conditions = []
    conditions += [x_dreal[i] == x[0][i] for i in range(sys.nx)]
    conditions += [u_dreal[i] == u[0][i] for i in range(sys.nu)]
    conditions += [xd_dreal[i] <= xd[i]+1e-3 for i in range(sys.nx)]
    conditions += [xd_dreal[i] >= xd[i]-1e-3 for i in range(sys.nx)]
    fsat = d.And(
      *conditions
    )

    r = d.CheckSatisfiability(fsat, 0.001)
    print(r)


test_dreal_for_benchmark()

In [5]:
import numpy as np
import torch
from torch import nn
# import dreal as d

class TanhNetwork(nn.Module):
    def __init__(self, dims, final_act='tanh'):
        super().__init__()
        self.dims = dims
        self.final_act = final_act
        
        layers = []
        for i in range(len(dims)-2):
            layers.append( nn.Linear(dims[i], dims[i+1]) )
            layers.append( nn.Tanh() )
        
        layers.append( nn.Linear(dims[-2], dims[-1]) )
        if final_act == 'tanh':
            layers.append( nn.Tanh())
        elif final_act == 'sigmoid':
            layers.append( nn.Sigmoid() )
        else:
            raise "Not Implemented"
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layers(x).squeeze(-1)
    
    def forward_with_grad(self, x):
        """
        This function should only be called when dims[-1]=1
        :param x: [batch, input_dim]
        :return:
            y: [batch,]
            grad: [batch, input_dim], grad[i] = d/dx f(x_i)
        """
        assert self.dims[-1] == 1
        y = self(x)

        # grad f(x)
        jacob = torch.autograd.functional.jacobian(
            self,
            (x,),
            create_graph=True
        )
        jacob = jacob[0]  # [batch, batch, dim]
        grad = torch.diagonal(jacob).T  # diagonal(shape (3,3,2)) gives (2,3), but we want (3,2)
        
        return y, grad
    
    def get_param_pair(self):
        """get_param_pair
        
        :return:
        ws: Weight variables in relu network
        bs: Bias variables in relu network
        ws and bs must have the same length
        
        """
        ws = []
        bs = []
        
        for name, param in self.layers.named_parameters():
        
            if "weight" in name:
                # print(param.shape)
                ws.append(param.cpu().detach().numpy())
            elif "bias" in name:
                # print(param.shape)
                bs.append(param.cpu().detach().numpy())
        if len(bs) == 0:
            bs = [
                np.zeros([w.shape[0]]) for w in ws
            ]
        return ws, bs
    
    def forward_dreal(self, x):
        """
        Construct dreal expression for the neural network
        :param x: np.array[dreal.Variable]
        """
        ws, bs = self.get_param_pair()
        for w, b in zip(ws[:-1], bs[:-1]):
            x = dreal_elementwise(w @ x + b, d.tanh)

        x = ws[-1] @ x + bs[-1]
        if self.final_act == 'tanh':
            x = dreal_elementwise(x, d.tanh)
        elif self.final_act == 'sigmoid':
            x = dreal_elementwise(x, dreal_sigmoid)
        return x

In [ ]:
def test_dreal_for_network(act='tanh'):
    net = TanhNetwork([2, 1], act)
    x_dreal = dreal_var(2)
    y_dreal = net.forward_dreal(x_dreal)

    x = np.random.randn(1,2)
    y = torch_to_np(net(np_to_torch(x)))[0]

    print('x:', x, y)

    conditions = []
    conditions += [x_dreal[i] == x[0][i] for i in range(len(x))]
    conditions += [y_dreal[0] >= y - 1e-3, y_dreal[0] <= y + 1e-3] # avoid floating point issue in pytorch
    print(conditions)
    fsat = d.And(
      *conditions
    )

    r = d.CheckSatisfiability(fsat, 0.001)
    print(r)

test_dreal_for_network('tanh')
print('---')
test_dreal_for_network('sigmoid')

In [7]:
import os
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR

class Trainer():
    def __init__(self,
                 system:Benchmark,
                 w_dim,
                 c_dim, 
                 alpha=0.01,
                 batch_size=16,
                 path_sampled=8,
                 integ_threshold=200,
                 norm_threshold=1e-2,
                 dt=0.05,
                 learning_rate=1e-2):
        """
		:param system:
		:param w_dim: [nx, ..., 1]
		:param c_dim: [nx, ..., nu]
		:param alpha: hyperparameter
		:param batch_size: number of sample points for each iteration
		:param path_sampled: number of paths to sample for each iteration
		:param integ_threshold: see simulate_trajectory
		:param norm_threshold: see simulate_trajectory
		:param dt: time interval for RK-4
		"""
        if not os.path.exists('./%s' % system.name):    os.mkdir('./%s' % system.name)
        if not os.path.exists('./%s/plots' % system.name):  os.mkdir('./%s/plots' % system.name)
        if not os.path.exists('./%s/ckpts' % system.name):  os.mkdir('./%s/ckpts' % system.name)
        
        self.system = system
        self.controller = TanhNetwork(c_dim, final_act='tanh')
        self.W = TanhNetwork(w_dim, final_act='sigmoid')
        self.alpha = alpha
        self.batch_size = batch_size
        self.path_sampled = path_sampled
        self.integ_threshold = integ_threshold
        self.norm_threshold = norm_threshold
        self.dt = dt
        self.learning_rate = learning_rate
        self.zero = np_to_torch(np.zeros([1, system.nx]))
        
    
    def load_ckpt(self, fname):
        ckpt = torch.load(fname, map_location=device)
        self.W.load_state_dict(ckpt['W'])
        self.controller.load_state_dict(ckpt['C'])
        
    @torch.no_grad()
    def simulate_trajectory(self, x=None, max_steps=int(1e7)):
        """
        Simulate a trajectory, terminate the forward rollout if
			1. the norm is less than self.norm_threshold
			2. the number of steps exceeds self.integ_threshold
			3. the trajectory stabilizes
			4. the number of steps exceeds max_steps
					
        :param x: torch.tensor with shape [1, nx] or None
            the trajectory starts at x, if x is None, the starting point is randomly sampled
        :param max_steps: 

        :return
            traj: [steps, nx]
            integ: norm integration from the starting point, float
            convergence: True, (only false if 2nd condition is satisfied, this is for ploting diverging trajectories)
        """
        integ_acc = 0.
        steps = 0
        
        x = np_to_torch(self.system.sample_in_domain(1)) if x is None else x
        x_hist = [x.clone()]
        
        while True:
            steps += 1
            norm = torch.linalg.norm(x).item()
            integ_acc += norm * self.dt
            
            if norm < self.norm_threshold \
            or ( len(x_hist) > 10 and torch.linalg.norm(x_hist[-1] - x_hist[-10]) < 1e-3 ) \
            or steps >= max_steps:
                return torch.cat(x_hist, dim=0), integ_acc, True
            
            elif integ_acc > self.integ_threshold:
                return torch.cat(x_hist, dim=0), integ_acc, False
            
            u = self.controller(x)
            if self.system.nu == 1:
                u = u[:, None]
            x = rk4_step(self.system.f_torch, x_hist[-1], u, dt=self.dt)
            x_hist.append(x.clone())
            
    def train(self, iterations=2000):
        num_iter = 0
        optimizer = torch.optim.Adam(
			list(self.W.parameters()) + list(self.controller.parameters()),
			lr=self.learning_rate
		)
        scheduler = StepLR(optimizer, step_size=500, gamma=0.8)
        
        # plotting, only for 2d data
        xx_plot = np.linspace(self.system.lb[0] * 2, self.system.ub[0] * 2, 3000)
        yy_plot = np.linspace(self.system.lb[1] * 2, self.system.ub[1] * 2, 3000)
        X_plot, Y_plot = np.meshgrid(xx_plot, yy_plot)
        xys_plot = np.stack([X_plot, Y_plot], axis=-1).reshape([-1, 2])
        xys_plot = np_to_torch(xys_plot)
        
        for unused in range(iterations):
            num_iter += 1
            vs = []
            xs = []
            
            for _ in range(self.path_sampled):
                traj, integ, _ = self.simulate_trajectory()
                xs.append(traj[0])
                vs.append(integ)
            xs = torch.stack(xs, dim=0)
            vs = np_to_torch(np.array(vs))
            
            critic_loss = 0.
            actor_loss = 0.
            
            # W(0) = 0
            W0 = self.W(self.zero)
            # critic_loss += 5. * torch.square(W0) # pendulum
            critic_loss += 5. * torch.square(W0) # tracking
            
            # Approximate W(x) = tanh(alpha * V(x))
            Wx = self.W(xs)
            What = torch.tanh(self.alpha * vs)
            critic_loss += F.mse_loss(Wx, What)
            
            
            # Physics-Informed Loss (PDE residual)
            # xs = np_to_torch(self.system.sample_in_domain(self.batch_size, scale=1. if num_iter < 1000 else 2.))
            xs = np_to_torch(self.system.sample_in_domain(self.batch_size, scale=1.))
            Wx, grad_Wx = self.W.forward_with_grad(xs)
            us = self.controller(xs)
            if self.system.nu == 1:
                us = us[:, None]
            fx = self.system.f_torch(xs, us)
            xnorm = torch.norm(xs, p=2, dim=1)
            residual = torch.sum(grad_Wx * fx.detach(), dim=1) + self.alpha * xnorm * (1 + Wx) * (1 - Wx)
            critic_loss += 1. * torch.mean(torch.square(residual))
            
            
            # Controller Loss
            grad_Wx = grad_Wx / torch.linalg.norm(grad_Wx, dim=1, keepdim=True)
            actor_loss += 1. * torch.mean(
                torch.sum(grad_Wx.detach() * fx, dim=1)
            )

            
            # Barrier
            # if num_iter < 1000:
            xs = np_to_torch(self.system.sample_out_of_domain(self.batch_size, scale=2.))
            Wx = self.W(xs)
            
            # critic_loss += 2. * F.l1_loss(Wx, torch.ones_like(Wx).to(device)) # pendulum
            critic_loss += 2. * F.l1_loss(Wx, torch.ones_like(Wx).to(device)) # tracking
                
            loss = .5 * actor_loss + critic_loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            if num_iter % 10 == 0:
                print(num_iter, critic_loss.item(), actor_loss.item())
            
            if num_iter % 20 == 0:
                vals = pipe(xys_plot, self.W, torch_to_np)
                fig = plt.figure()
                ax = plt.axes()
                im = ax.contourf(X_plot, Y_plot, np.reshape(vals, [len(xx_plot), len(yy_plot)]), levels=100)
                fig.colorbar(im)
                
                for _ in range(5):
                    path, _, convergence = self.simulate_trajectory(max_steps=3000)

                    if convergence:
                        path = torch_to_np(path)
                        plt.plot(path[0, 0], path[0, 1], 'r+')
                        plt.plot(path[:, 0], path[:, 1], 'o-', markersize=1, linewidth=0.5)
                plt.gca().set_aspect('equal')
                plt.savefig('./%s/plots/%d.png' % (self.system.name, num_iter))
                plt.close()
                
            if num_iter % 1000 == 0:
                torch.save(
                    {
                        'W': self.W.state_dict(),
                        'C': self.controller.state_dict()
                    },
                    './%s/ckpts/%d.pth' % (self.system.name, num_iter)
                )
                
    def check_lyapunov(self, level=0.9, scale=2., eps=0.5):
        W0 = self.W(self.zero).squeeze().item()
        x = dreal_var(self.system.nx)
        x_norm = d.Expression(0.)
        lie_derivative_W = d.Expression(0.)

        # construct xnorm and f(x, u)^T \nabla_x W(x)
        u = self.controller.forward_dreal(x)
        fx = self.system.f_dreal(x, u)
        Wx = self.W.forward_dreal(x)[0]

        # construct x_norm and <fx, \grad_x W(x)>
        for i in range(self.system.nx):
            x_norm += x[i] * x[i]
            lie_derivative_W += fx[i] * Wx.Differentiate(x[i])
        x_norm = d.sqrt(x_norm)

        condition = d.And(
            x_norm >= eps,
            self.system.in_domain_dreal(x, scale),
            Wx <= level,
            d.Or(
                lie_derivative_W >= 0,
                Wx <= W0
            )
        )
        r1 = d.CheckSatisfiability( condition, 0.0001 )
        
        r2 = d.CheckSatisfiability(
            d.And(
                self.system.on_boundry_dreal(x, scale=scale),
                Wx <= level
            ),
            0.0001
        )
        return r1, r2

# Training Double Integrator

In [172]:
# system = DoubleIntegrator()
# trainer = Trainer(system,
#                   [system.nx, 20, 20, 1],
#                   [system.nx, 10, 10, system.nu],
#                   alpha=0.05,
#                   dt=0.01,
#                   norm_threshold=1e-2,
#                   integ_threshold=150,
#                   batch_size=64,
#                   path_sampled=8,
#                   learning_rate=2e-3)
# trainer.train(3000)
# trainer.check_lyapunov(0.7, 2.0, 0.5)

# Training Vander Pol

In [ ]:
# The training is slow for the first (roughly) 80 iterations since the dynamics forms an orbit and the path sampling always exceeds the maximum steps.


# vanderpol
# system = VanderPol()
# trainer = Trainer(system,
#                   [system.nx, 30, 30, 1],
#                   [system.nx, 30, 30, system.nu],
#                   alpha=0.1,
#                   dt=0.01,
#                   norm_threshold=1e-2,
#                   integ_threshold=50,
#                   batch_size=64,
#                   path_sampled=8,
#                   learning_rate=3e-3)
# 
# trainer.train(3000)
# trainer.check_lyapunov(0.5, 2.0, 0.5)

# Training Inverted Pendulum

In [68]:
# system = InvertedPendulum()
# trainer = Trainer(system,
#                   [system.nx, 20, 20, 1],
#                   [system.nx, 5, 5, system.nu],
#                   alpha=0.2,
#                   dt=0.003,
#                   norm_threshold=1e-2,
#                   integ_threshold=150,
#                   batch_size=64,
#                   path_sampled=8,
#                   learning_rate=2e-3)
# trainer.train(3000)
# trainer.check_lyapunov(0.7, 2.0, 0.5)

# Training Bicycle Tracking

In [ ]:
# system = BicycleTracking()
# trainer = Trainer(system,
#                   [system.nx, 20, 20, 1],
#                   [system.nx, 10, 10, system.nu],
#                   alpha=1.5,
#                   dt=0.003,
#                   norm_threshold=1e-2,
#                   integ_threshold=50,
#                   batch_size=64,
#                   path_sampled=8,
#                   learning_rate=2e-3)
# trainer.train(3000)
# trainer.check_lyapunov(0.4, 2.0, 0.2)

In [ ]:
x = np_to_torch(np.array([[-0.083125000000000004,1.]]))
Wx, gradWx = trainer.W.forward_with_grad(x)
u = trainer.controller(x)[:,None]
fx = trainer.system.f_torch(x, u)
print(Wx)
print(torch.sum(gradWx * fx, dim=1))
print(trainer.W(trainer.zero))